# HW05: базовый ML-пайплайн с логистической регрессией и сравнение с бейзлайном

Домашнее задание: загрузка данных, базовая разведка, подготовка признаков, бейзлайн через `DummyClassifier`, логистическая регрессия с подбором `C`, сравнение метрик и краткие выводы.


## 2.3.1. Загрузка данных и первичный анализ

Импортируем библиотеки, читаем S05-hw-dataset.csv и смотрим базовые характеристики датасета: размер, типы столбцов, описательные статистики и баланс таргета `default`.


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn import metrics
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

sns.set(style="whitegrid")
pd.set_option("display.max_columns", 50)

DATA_PATH = Path("S05-hw-dataset.csv")
FIGURES_DIR = Path("figures/")
FIGURES_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
df = pd.read_csv(DATA_PATH)

print(f"Датасет загружен из {DATA_PATH}")
print(f"Размер: {df.shape[0]} объектов, {df.shape[1]} столбцов")

display(df.head())
print("\nИнформация о столбцах:")
print(df.info())

print("\nОписательные статистики (числовые фичи):")
display(df.describe().T)

print("\nБаланс таргета default:")
value_counts = df["default"].value_counts()
value_share = df["default"].value_counts(normalize=True)
display(pd.DataFrame({"count": value_counts, "share": value_share}))


In [ ]:
n_rows, n_cols = df.shape
non_numeric_cols = [col for col in df.columns if not pd.api.types.is_numeric_dtype(df[col])]
debt_bounds = df["debt_to_income"].agg(["min", "max"])
neg_savings = int((df["savings_balance"] < 0).sum())
neg_checking = int((df["checking_balance"] < 0).sum())

print(f"Количество объектов: {n_rows}, число столбцов (включая таргет): {n_cols}")
print(f"Ненумерических столбцов: {len(non_numeric_cols)} -> {non_numeric_cols}")
print(f"Диапазон debt_to_income: min={debt_bounds['min']:.4f}, max={debt_bounds['max']:.4f}")
print(f"Отрицательные значения в savings_balance: {neg_savings}")
print(f"Отрицательные значения в checking_balance: {neg_checking}")


In [ ]:
target_counts = df["default"].value_counts().to_dict()
target_share = df["default"].value_counts(normalize=True).to_dict()
summary_text = f"""
**Краткие наблюдения**
- В выборке {n_rows} объектов и {n_cols - 1} признаков после исключения таргета.
- Все столбцы числовые, дополнительная кодировка категорий не требуется.
- Диапазон `debt_to_income` лежит в [{debt_bounds['min']:.3f}; {debt_bounds['max']:.3f}], выглядит допустимо.
- Отрицательные остатки: savings_balance={neg_savings}, checking_balance={neg_checking}; можно считать это допустимыми овердрафтами, явных аномалий не видно.
- Баланс классов: default=0 — {target_counts.get(0, 0)} ({target_share.get(0, 0):.3f}), default=1 — {target_counts.get(1, 0)} ({target_share.get(1, 0):.3f}).
"""

display(Markdown(summary_text))


## 2.3.2. Подготовка признаков и таргета

Таргет — столбец `default`. В качестве признаков используем все остальные столбцы, исключая `client_id`. Проверим, что признаки числовые, и контролируем диапазон `debt_to_income`.


In [ ]:
feature_cols = df.columns.drop(["default", "client_id"])
X = df[feature_cols]
y = df["default"]

non_numeric_after = [col for col in X.columns if not pd.api.types.is_numeric_dtype(X[col])]
debt_ok_share = ((X["debt_to_income"] >= 0) & (X["debt_to_income"] <= 1)).mean()

print(f"Признаков в X: {X.shape[1]}, объектов: {X.shape[0]}")
print(f"Ненумерические признаки: {non_numeric_after}")
print(f"Доля объектов с debt_to_income в [0,1]: {debt_ok_share:.3f}")


## 2.3.3. Train/Test-сплит и бейзлайн-модель

Делим данные на обучение/тест с стратификацией по таргету, строим `DummyClassifier` как точку отсчёта и считаем метрики accuracy и ROC-AUC.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42,
)

print(
    f"Train shape: {X_train.shape}, Test shape: {X_test.shape},"
    f" positive share train={y_train.mean():.3f}, test={y_test.mean():.3f}"
)


In [ ]:
dummy = DummyClassifier(strategy="stratified", random_state=42)
dummy.fit(X_train, y_train)

y_pred_dummy = dummy.predict(X_test)
y_proba_dummy = dummy.predict_proba(X_test)[:, 1]

baseline_metrics = {
    "model": "DummyClassifier",
    "accuracy": metrics.accuracy_score(y_test, y_pred_dummy),
    "roc_auc": metrics.roc_auc_score(y_test, y_proba_dummy),
    "precision": metrics.precision_score(y_test, y_pred_dummy),
    "recall": metrics.recall_score(y_test, y_pred_dummy),
    "f1": metrics.f1_score(y_test, y_pred_dummy),
}

print("Бейзлайн (DummyClassifier, stratified):")
display(pd.DataFrame([baseline_metrics]))
print("DummyClassifier просто имитирует распределение классов, выступая точкой отсчёта для более сложных моделей.")


## 2.3.4. Логистическая регрессия и подбор гиперпараметров

Строим конвейер StandardScaler → LogisticRegression, подбираем `C` через GridSearchCV и оцениваем лучшую модель по accuracy, ROC-AUC и дополнительным метрикам. Построим ROC-кривую и сохраним её в `homeworks/HW05/figures/`.


In [ ]:
logreg_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(max_iter=1000, solver="lbfgs")),
])

param_grid = {
    "logreg__C": [0.01, 0.1, 1.0, 10.0, 100.0],
}

grid = GridSearchCV(
    estimator=logreg_pipe,
    param_grid=param_grid,
    cv=5,
    scoring="roc_auc",
    n_jobs=-1,
)

grid.fit(X_train, y_train)

best_model = grid.best_estimator_
print(f"Лучшая модель по ROC-AUC на кросс-валидации: {grid.best_params_}")


In [ ]:
y_pred_lr = best_model.predict(X_test)
y_proba_lr = best_model.predict_proba(X_test)[:, 1]

logreg_metrics = {
    "model": "LogisticRegression",
    "best_C": grid.best_params_["logreg__C"],
    "accuracy": metrics.accuracy_score(y_test, y_pred_lr),
    "roc_auc": metrics.roc_auc_score(y_test, y_proba_lr),
    "precision": metrics.precision_score(y_test, y_pred_lr),
    "recall": metrics.recall_score(y_test, y_pred_lr),
    "f1": metrics.f1_score(y_test, y_pred_lr),
}

conf_mat = metrics.confusion_matrix(y_test, y_pred_lr)

print("Метрики LogisticRegression (лучшая по GridSearchCV):")
display(pd.DataFrame([logreg_metrics]))
print("Confusion matrix (rows=true, cols=pred):")
display(pd.DataFrame(conf_mat, index=["true_0", "true_1"], columns=["pred_0", "pred_1"]))


In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba_lr)
roc_auc_lr = metrics.roc_auc_score(y_test, y_proba_lr)

plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"LogReg (AUC={roc_auc_lr:.3f})")
plt.plot([0, 1], [0, 1], "--", color="gray", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-кривая: Logistic Regression")
plt.legend()
plt.tight_layout()

roc_path = FIGURES_DIR / "roc_curve_logreg.png"
plt.savefig(roc_path, dpi=150)
plt.show()
print(f"ROC-кривая сохранена в {roc_path}")


## 2.3.5. Сравнение моделей и текстовые выводы

Сведём метрики бейзлайна и логистической регрессии, а затем сформулируем итоговые выводы по качеству и влиянию регуляризации.


In [ ]:
baseline_metrics.setdefault("best_C", np.nan)
results_df = pd.DataFrame([baseline_metrics, logreg_metrics])[
    ["model", "best_C", "accuracy", "roc_auc", "precision", "recall", "f1"]
]

print("Сводная таблица метрик:")
display(results_df)


In [ ]:
delta_acc = logreg_metrics["accuracy"] - baseline_metrics["accuracy"]
delta_auc = logreg_metrics["roc_auc"] - baseline_metrics["roc_auc"]

summary_md = f"""
**Итоговые выводы**
1. Бейзлайн DummyClassifier (strategy=stratified) даёт accuracy {baseline_metrics['accuracy']:.3f} и ROC-AUC {baseline_metrics['roc_auc']:.3f}; метрики соответствуют случайным угадываниям, что ожидаемо.
2. Логистическая регрессия с лучшим `C={logreg_metrics['best_C']}` показывает accuracy {logreg_metrics['accuracy']:.3f} и ROC-AUC {logreg_metrics['roc_auc']:.3f}, превосходя бейзлайн.
3. Прирост accuracy = {delta_acc:.3f}, прирост ROC-AUC = {delta_auc:.3f}; выигрыш особенно заметен по AUC, что важно при несбалансированных классах.
4. Значение `C={logreg_metrics['best_C']}` говорит, что модель предпочла {"сильную" if logreg_metrics['best_C'] < 1 else "умеренную" if logreg_metrics['best_C'] == 1 else "более слабую"} регуляризацию; слишком маленькие C ухудшали качество, слишком большие — не дали дополнительного выигрыша.
5. Precision/recall {logreg_metrics['precision']:.3f}/{logreg_metrics['recall']:.3f} выше, чем у бейзлайна, а матрица ошибок показывает лучшее распознавание положительного класса.
6. ROC-кривая логистической регрессии выше диагонали, файл сохранён в папке `homeworks/HW05/figures/` и подтверждает стабильное превосходство над случайным угадыванием.
7. Для задачи кредитного скоринга логистическая регрессия выглядит разумным выбором: метрики лучше, интерпретируемость высокая, пайплайн простой и воспроизводимый.
"""

display(Markdown(summary_md))
